# Gini 네비게이션의 Last.bin파일 분석

+ Gini 네비게이션에서 사용자의 마지막 위치를 기록하는 Last.bin파일을 분석함. 
+ Last.bin 파일은 UserData/Rec에 존재.

In [1]:
import struct
import folium

### 메소드 설명
+ Gini 네비게이션 상의 Lats.bin 파일의 데이터 분석을 위한 함수 정의
+ 바이너리 파일의 앞의 8바이트가 위도와 경도를 결정하며, 아래의 함수의 식을 통해 위도와 경도를 반환받음.

    + 위도와 경도는 파일의 첫 8바이트에서 결정됨, 

    ```python
        X_Line = data[0:4]  ## 경도를 결정
        Y_Line = data[4:8]  ## 위도를 결정
    ```

    + 각각의 데이터는 리틀엔디언 형식으로 작성되어 있음.
    + 따라서, strcut 라이브러리의 unpack 함수를 이용하여 '<' 파라미터를 통해리틀 앤디언 형식의 bytearray를 10진수로 변환함.

    ```python
        X_Line = struct.unpack('<HH',X_Line) 
    ```

    + 최대 2바이트 단위로 인코딩 하므로, 아래의 식을 통해 int로 변환함

    ```python
        X_Line = (X_Line[1]<<16)+X_Line[0]
    ```

    + 인용한 논문에 따르면, 지니 네비게이션의 경우 아래의 변환식을 통해 좌표값을 변환해야함.
        + X축 값 = x + 81,284,000
        + Y축 값 = y + 23,820,000
        

In [2]:
def GiniLastbin(data):
    X_Line = data[0:4]
    Y_Line = data[4:8]
    
    X_Line = struct.unpack('<HH',X_Line)
    X_Line = (X_Line[1]<<16)+X_Line[0]
    
    Y_Line = struct.unpack('<HH',Y_Line)
    Y_Line = (Y_Line[1]<<16)+Y_Line[0]
    
    X_Line += 81284000
    Y_Line += 23820000
    
    return Y_Line,X_Line  ## (위도, 경도)

In [3]:
Last_Data = open('./Last.bin', 'rb')
data = Last_Data.read()
Last_Data.close()

In [45]:
GiniLastbin(data)

(37238384, 127046633)

+ Last.bin 파일의 데이터를 위의 GiniLastbin 함수를 통해 반환한 결과를 지도에 표현함.

In [50]:
m = folium.Map([GiniLastbin(data)[0]/10**6,GiniLastbin(data)[1]/10**6], zoom_start = 15)
folium.Marker([GiniLastbin(data)[0]/10**6,GiniLastbin(data)[1]/10**6],tooltip = 'Last.bin에 저장된 장소').add_to(m)
m

참고문헌: 최용석. "디지털 포렌식 관점에서의 내비게이션 사용흔적 정보 분석." 국내석사학위논문 高麗大學校 情報經營工學專門大學院, 2010. 서울